[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, emoji, collections, IPython, mistune, doctest, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    with __import__('importnb').Notebook():
        try:
            from . import tangle
        except:
            from pidgin import tangle
            
    with tangle.Pidgin(): 
        try:
            from . import shell
            from .specifications import template, markdown, yaml, transform_cell, emojis, json
        except:
            from pidgin import shell
            from pidgin.specifications import template, markdown, yaml, transform_cell, emojis, json

    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    def load_ipython_extension(shell: IPython.InteractiveShell):
>>> assert '_document_' in sys.modules
>>> assert '_test_' in sys.modules
        
        from . import testing, template
        document, test = map(make_module, '_document_ _test_'.split())
        test = shell.test = testing.Doctest(user_module=test, parent=shell, parents=[shell])
        shell.markdown = markdown.Markdown(user_module=document, parent=shell, parents=[test, shell])
        template = shell.template = template.Template(parent=shell, parents=[shell.markdown, test, shell])
        shell.weave = Weave(user_module=document, parent=shell, parents=[test, shell], markdown=shell.markdown, template=template, test=test, )
        
        shell.display_formatter.formatters['text/html'].for_type(type(shell), format_http)
        shell.events.register('post_run_cell', test)
        shell.events.register('post_run_cell', shell.weave)
        shell.enable_html_pager = True        

In [3]:
    def format_http(str): 
        if str.startswith(('http:', 'https:')): return mistune.markdown(str)

In [4]:
    class Weave(template.Template, yaml.FrontMatter, shell.Shell):
        test = traitlets.Instance(IPython.InteractiveShell)
        weave = traitlets.Instance(IPython.InteractiveShell)
        markdown = traitlets.Instance(IPython.InteractiveShell, allow_none=True)
        template = traitlets.Instance(IPython.InteractiveShell)
        
        __call__ = shell.Shell.__call__
        
        def __init__(Weave, *args, **kwargs):
            
            shell.Shell.__init__(Weave, *args, **kwargs)
            yaml.FrontMatter.__init__(Weave)
            
        def run_cell(Weave, text: str, *args, **kwargs):
            if isinstance(text, IPython.core.interactiveshell.ExecutionResult): 
                text = text.info.raw_cell
                                
            publish = do_publish(''.join(text))
            text = text.lstrip('//')
            if text in Weave.user_ns:
                data, metadata = Weave.parent.display_formatter.format(Weave.user_ns[text])
                Weave.parent.display_pub.publish(data, metadata, transient={'raw': True})
                return 
            text, metadata = Weave.front_matter(text)
            
            text = json.expand(text, Weave.user_ns.get('__context__', {}))
            if len(text.splitlines()) == 1 and text.startswith(('http:', 'https:')):
>>> __context__ = """---
... w: https://en.wikipedia.org/wiki/"""
>>> with IPython.utils.capture.capture_output() as out:
...     get_ipython().weave.run_cell("w:pidgin")
>>> ' '.join(out.outputs[0].data['text/html'].strip().split())
'<iframe width="100%" height="600px" src="https://en.wikipedia.org/wiki/pidgin" frameborder="0" allowfullscreen ></iframe>'

                
                data, metadata = Weave.parent.display_formatter.format(IPython.display.IFrame(text, "100%", "600px"))
            else:
                text = emojis.emojize(text)
                text = template.Template.render(Weave, text, **metadata)
                text = Weave.markdown.render(text)                
>>> with IPython.utils.capture.capture_output() as out:
...     get_ipython().weave.run_cell("# Any markdown you want {{'___smiling_face___' | red}}")
>>> out.outputs[0].data
{'text/plain': '<IPython.core.display.HTML object>', 'text/html': '<h1>Any markdown you want <span style="color:#ff0000;">☺</span></h1>\n'}

                if publish:
                    data, metadata = Weave.parent.display_formatter.format(IPython.display.HTML(text, metadata=metadata))
                else: 
>>> with IPython.utils.capture.capture_output() as out:
...     get_ipython().weave.run_cell("\n# Any markdown you want {{'___smiling_face___' | red}}")
>>> assert not out.outputs

>>> with IPython.utils.capture.capture_output() as out:
...     get_ipython().weave.run_cell("//# Any markdown you want {{'___smiling_face___' | red}}")
>>> assert not out.outputs

                    
                    data = {}
            
            (data or metadata) and Weave.parent.display_pub.publish(data, metadata)


In [5]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
>>> assert only_contains_block_code("""f""", """f""")
>>> assert only_contains_block_code("""%%magic""", """%%magic""")
>>> assert not only_contains_block_code("""Markdown""", """'Markdown'""")


        
        return raw.startswith('%%') or list(map(str.rstrip, textwrap.dedent(raw).splitlines())) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

In [6]:
    def do_publish(str) -> bool:
>>> assert do_publish("""___beach_with_umbrella___""")
>>> assert not do_publish("""//___bear_face___""")
>>> assert not do_publish("""
... ___no_bicycles___""")
        
        display = bool(str.partition('\n')[0].strip()) and not str.startswith('//')            
        source = transform_cell.markdown_to_python(str)
        display = display & (not bool(only_contains_block_code(str, source)))
        try:
            if str.strip() != ast.literal_eval(source.rstrip().rstrip(';')).strip(): display = False
        finally: return display

In [7]:
    def make_module(str):
>>> make_module('_an_opportunistic_module_')
<module '_an_opportunistic_module_'>
>>> assert '_an_opportunistic_module_' in sys.modules

        
        try:  return importlib.import_module(str)
        except:  return sys.modules.__setitem__(str, types.ModuleType(str)) or sys.modules[str]

In [8]:
    def unload_ipython_extension(shell: IPython.InteractiveShell):
        from . import testing
        pops = []
        for i, post in enumerate(shell.events.callbacks['post_run_cell']):
            if isinstance(post, (testing.Doctest, Weave)):
                pops.append(i)
        for i in reversed(pops): shell.events.callbacks['post_run_cell'].pop(i)